# Neural Network 神经网络


## 加载训练集并预处理

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.io as sio
import matplotlib

In [2]:
# 加载数据的函数
def load_data(path):
    data = sio.loadmat(path)
    X = data.get('X')
    y = data.get('y')  # shape为二位数组 (5000,1)
    y = y.reshape(y.shape[0])
    return X, y

- 意思： sio 是 scipy.io 的缩写。loadmat 专门用来读取 MATLAB 保存的 .mat 文件。
- 结果： data 变成了一个字典（Dictionary），里面装了 .mat 文件里的所有变量。
- 返回的是numpy数组类型

In [3]:
def sigmoid(z) :
    return 1 / (1 + np.exp(-z))

# 神经网络模型图示
![出错了！！！](https://github.com/66666-visit/Machine-learning/blob/main/%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C/%E5%89%8D%E5%90%91%E4%BC%A0%E6%92%AD/nn_model.png?raw=true)


In [16]:
def load_weight(path):
    data = sio.loadmat(path)
    return data['Theta1'], data['Theta2']


In [17]:
theta1, theta2 = load_weight('ex3weights.mat')
theta1.shape, theta2.shape
print(theta1)
print(theta2)

[[-2.25623899e-02 -1.05624163e-08  2.19414684e-09 ... -1.30529929e-05
  -5.04175101e-06  2.80464449e-09]
 [-9.83811294e-02  7.66168682e-09 -9.75873689e-09 ... -5.60134007e-05
   2.00940969e-07  3.54422854e-09]
 [ 1.16156052e-01 -8.77654466e-09  8.16037764e-09 ... -1.20951657e-04
  -2.33669661e-06 -7.50668099e-09]
 ...
 [-1.83220638e-01 -8.89272060e-09 -9.81968100e-09 ...  2.35311186e-05
  -3.25484493e-06  9.02499060e-09]
 [-7.02096331e-01  3.05178374e-10  2.56061008e-09 ... -8.61759744e-04
   9.43449909e-05  3.83761998e-09]
 [-3.50933229e-01  8.85876862e-09 -6.57515140e-10 ... -1.80365926e-06
  -8.14464807e-06  8.79454531e-09]]
[[-0.76100352 -1.21244498 -0.10187131 -2.36850085 -1.05778129 -2.20823629
   0.56383834  1.21105294  2.21030997  0.44456156 -1.18244872  1.04289112
  -1.60558756  1.30419943  1.37175046  1.74825095 -0.23365648 -1.52014483
   1.15324176  0.10368082 -0.37207719 -0.61530019 -0.1256836  -2.27193038
  -0.71836208 -1.29690315]
 [-0.61785176  0.61559207 -1.26550639  1.

###  这就是那个“训练好的大脑”。
1. theta1 (25, 401)： 这是第一层权重。意味着有 25 个“特征侦探”，每个侦探都要看那 401 个输入特征。
2. theta2 (10, 26)： 这是第二层权重。意味着有 10 个“最终裁判”，每个裁判都要听取上一层侦探的汇报。注意这里是 26！ (为什么是 26？往下看)

### 加载数据

In [6]:
X, y = load_data('ex3data1.mat')
X = np.insert(X, 0, values=np.ones(X.shape[0]), axis=1)  # intercept
X.shape, y.shape

((5000, 401), (5000,))

1. 这里加载的 X 是 5000 张图片。每张图本来是 20x20 的方块，但已经被“压扁”成了一条包含 400 个数字的长条。
2. 关键动作 np.insert： 你看代码里，它在 X 的第 0 列硬塞了一排 1。
3. 为什么要塞 1？ 这叫 截距项 (Bias Unit)。你可以把它想象成“基础分”或者“起步价”。如果不加这一列，矩阵乘法时维度就对不上，且神经元无法进行平移调整。
4. 现在的形状： X 变成了 (5000, 401)。意思是：5000 个考生，每人拿着 401 个特征（400个像素 + 1个基础分）进考场。

# Feed Forward Prediction（前馈预测）

In [7]:
a1 = X

In [8]:
z2 = a1 @ theta1.T # (5000, 401) @ (25,401).T = (5000, 25)
z2.shape

(5000, 25)

In [15]:
z2 = np.insert(z2, 0, values=np.ones(z2.shape[0]), axis=1)

In [10]:
a2 = sigmoid(z2)
a2.shape

(5000, 26)

a1 @ theta1.T 是什么意思？

1. 这是让那 25 个“侦探”去给 5000 张图打分。

   - 比如“侦探1”可能专门负责看图片里有没有“圆圈”。如果输入有圆圈，他和对应的像素权重一乘，得分就高。

   - 结果 z2： 变成了 (5000, 25)。现在的 5000 张图，不再是像素点了，而是变成了 25 个特征得分。

2. 为什么要再次 np.insert？

   - 这是新手最容易漏的一步！

   - 隐藏层的这 25 个结果要传给下一层时，同样需要一个“起步价”（截距项）。

   - 所以 z2 变成了 (5000, 26)。（原来的25个特征 + 1个刚加的截距）。

3. sigmoid 是干嘛的？

   - 它把得分（可能是 100，也可能是 -500）压缩到 0 到 1 之间。

   - 这就是我们在模拟神经元的“激活”状态。

In [11]:
z3 = a2 @ theta2.T
z3.shape

(5000, 10)

In [12]:
a3 = sigmoid(z3)
a3

array([[1.38245045e-04, 2.05540079e-03, 3.04012453e-03, ...,
        4.91017499e-04, 7.74325818e-03, 9.96229459e-01],
       [5.87756717e-04, 2.85026516e-03, 4.14687943e-03, ...,
        2.92311247e-03, 2.35616705e-03, 9.96196668e-01],
       [1.08683616e-04, 3.82659802e-03, 3.05855129e-02, ...,
        7.51453949e-02, 6.57039547e-03, 9.35862781e-01],
       ...,
       [6.27824726e-02, 4.50406476e-03, 3.54510925e-02, ...,
        2.63669734e-03, 6.89448164e-01, 2.74369466e-05],
       [1.01908736e-03, 7.34360211e-04, 3.78558700e-04, ...,
        1.45616578e-02, 9.75989758e-01, 2.33374461e-04],
       [5.90807037e-05, 5.41717668e-04, 2.58968308e-05, ...,
        7.00508308e-03, 7.32814653e-01, 9.16696059e-02]], shape=(5000, 10))

- 讲解：

  - 逻辑和上一步一模一样！

  - 10 个裁判（代表数字 0-9）根据 26 个线索（25个特征 + 1个截距）进行加权打分。

  - 最终结果 a3 (5000, 10)：

    - 这不再是特征了，而是 概率。

    - 每一行有 10 个小数。比如第一行可能是 [0.01, 0.99, 0.05, ...]。

    - 这代表模型认为：这第一张图，是数字 0 的概率是 1%，是数字 1 的概率是 99%...

In [ ]:
y_pred = np.argmax(a3, axis=1) + 1 
y_pred.shape

<class 'numpy.ndarray'>


- 老师讲解：
  - np.argmax(a3, axis=1)： 这句话的意思是——“请帮我找出每一行里，数值最大的那个数，排在第几号位置（索引）？”
  - 如果第 0 号位置概率最大，返回 0。
  - 如果第 5 号位置概率最大，返回 5。

# 准确率
 
虽然人工神经网络是非常强大的模型，但训练数据的准确性并不能完美预测实际数据，在这里很容易过拟合。

In [14]:
accuracy = np.sum( y_pred == y ) / len(y) * 100 
print(f"accuracy == {accuracy}%")


accuracy == 97.52%
